> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

# Exercise Set 9: Parsing and Information Extraction

*Morning, August 17, 2018*

In this Exercise Set we shall develop our webscraping skills even further by practicing navigating html trees using BeautifoulSoup and furthermore train extracting information from raw text with no html tags to help, using regular expressions. 

## Exercise Section 9.1: Parsing a Table from HTML using BeautifulSoup.

Yesterday I showed you a neat little prepackaged function in pandas that did all the work. However today we should learn the mechanics of it. *(It is not just for educational purposes, sometimes the package will not do exactly as you want.)*

We hit the Basketball stats page from yesterday again: https://www.basketball-reference.com/leagues/NBA_2018.html.


> **Ex. 9.1.1:** Here we practice simply locating the table node of interest using the `find` method build into BeautifoulSoup. But first we have to fetch the HTML using the `requests` module. Parse the tree using `BeautifulSoup`. And then use the **>Inspector<** tool (* right click on the table < press inspect element *) in your browser to see how to locate the Eastern Conference table node - i.e. the *tag* name of the node, and maybe some defining *attributes*.

In [ ]:
# [Answer to Ex. 9.1.1]

In [1]:
## Solution
from bs4 import BeautifulSoup
import requests

url = 'https://www.basketball-reference.com/leagues/NBA_2018.html'

response = requests.get(url)

html = response.text
soup = BeautifulSoup(html,'lxml')

table_node = soup.find('table',attrs={'id':'confs_standings_E'}) # find table node with specific attribute

You have located the table should now build a function that starts at a "table node" and parses the information, and outputs a pandas DataFrame. 

Inspect the element either within the notebook or through the **>Inspector<** tool and start to see how a table is written in html. Which tag names can be used to locate rows? How will you iterate through columns. Were is the header located?

> **Ex. 9.1.2:** First you parse the header which can be found in the canonical tag name: thead. 
Next you use the `find_all` method to search for the tag, and iterate through each of the elements extracting the text, using the `.text` method builtin to the the node object. Store the header values in a list container. 

In [ ]:
# [Answer to Ex. 9.1.2]

In [2]:
import pandas as pd
# Solution
name = table_node.caption.text
# parse header
header = table_node.thead.find_all('th') # locate each column name using the `th` tag, which entail a string .
# extract the label you want. brevity use .text, for a more informative get teh aria-label attribute
header = [col['aria-label'] for col in header]
print(header,len(header))

['Eastern Conference', 'Wins', 'Losses', 'Win-Loss Percentage', 'Games Behind', 'Points Per Game', 'Opponent Points Per Game', 'Simple Rating System'] 8


> **Ex. 9.1.3:** Next you locate the rows, using the canonical tag name: tbody. And from here you search for all rows tags. Fiugre out the tag name yourself, inspecting the tbody node in python or using the **Inspector**. 

In [ ]:
# [Answer to Ex. 9.1.3]

In [3]:
# parse rows: the canonical tbody locates the data body.
body = table_node.tbody
# rows are found using the "tr" tag
rows = body.find_all('tr')

> **Ex. 9.1.4:** Next run through all the rows and extract each value, similar to how you extracted the header. However here is a slight variation: Since each value node can have a different tag depending on whether it is a digit or a string, you should use the `.children` method instead of the `.find_all` - (or write compile a regex that matches both the td tag and the th tag.) 
>Once the value nodes of each row has been located using the `.children` method you should extract the value. Store the extracted rows as a list of lists: ```[[val1,val2,...valk],...]```

In [5]:
# [Answer to Ex. 9.1.4]

In [6]:
# each row value has different tags depending on their type (digit or string)
# function to check what tag it is and either convert to float or not. 
import numpy as np
def convert_value_type(value_node):
    if value_node.name == 'th':
        return value_node.text
    else: # assume node is td:
        try: 
            return float(value_node.text)
        except:
            return np.nan # assume there is no value if it fails. 
data = []
for row_node in rows:
    row = []
    for child in row_node.children:
        row.append(convert_value_type(child))
    data.append(row)


> **Ex. 9.1.5:** Convert the data you have collected into a pandas dataframe. _Bonus:_ convert the code you've written above into a function which scrapes the page for you and returns a dataframe. 

In [55]:
#[Answer 9.1.5]

In [7]:
# Simple answer
df = pd.DataFrame(data,columns=header)

# Bonus
def parse_html_table(table_node):
    # parse header
    header = table_node.thead.find_all('th') # locate each column name using the `th` tag, which entail a string .
    # extract the label you want. brevity use .text, for a more informative get teh aria-label attribute
    header = [col['aria-label'] for col in header]
    # parse rows: the canonical tbody locates the data body.
    body = table_node.tbody
    # rows are found using the "tr" tag
    rows = body.find_all('tr')
    # each row value has different tags depending on their type (digit or string)
    # function to check what tag it is and either convert to float or not. 
    import numpy as np
    def convert_value_type(value_node):
        if value_node.name == 'th':
            return value_node.text
        else: # assume node is td:
            try: 
                return float(value_node.text)
            except:
                return np.nan # assume there is no value if it fails. 
    data = []
    for row_node in rows:
        row = []
        for child in row_node.children:
            row.append(convert_value_type(child))
        data.append(row)
    df = pd.DataFrame(data,columns=header)
    return df

df = parse_html_table(table_node)
df.head()

,Eastern Conference,Wins,Losses,Win-Loss Percentage,Games Behind,Points Per Game,Opponent Points Per Game,Simple Rating System
0,Toronto Raptors* (1),59.0,23.0,0.720,NaN,111.7,103.9,7.29
1,Boston Celtics* (2),55.0,27.0,0.671,4.0,104.0,100.4,3.23
2,Philadelphia 76ers* (3),52.0,30.0,0.634,7.0,109.8,105.3,4.30
3,Cleveland Cavaliers* (4),50.0,32.0,0.610,9.0,110.9,109.9,0.59
4,Indiana Pacers* (5),48.0,34.0,0.585,11.0,105.6,104.2,1.18


> **Ex. 9.1.6:** Now locate all tables from the page, using the `.find_all` method searching for the table tag name. Iterate through the table nodes and apply the function created for parsing html tables. Store each table in a dictionary using the table name as key. The name is found by accessing the id attribute of each table node, using dictionary-style syntax - i.e. `table_node['id']`.

In [56]:
# [Answer to Ex. 9.1.6]

In [13]:
# Solution
tables = soup.find_all('table') # locate all table nodes
dfs = {}
for table_node in tables:
    name = table_node['id'] # access the id attribute. 
    table = parse_html_table(table_node) # apply parse_table function
    dfs[name] = table # store table in the dictionary
    
    
dfs

{'confs_standings_E':            Eastern Conference  Wins  Losses  Win-Loss Percentage  \
 0       Toronto Raptors* (1)   59.0    23.0                0.720   
 1        Boston Celtics* (2)   55.0    27.0                0.671   
 2    Philadelphia 76ers* (3)   52.0    30.0                0.634   
 3   Cleveland Cavaliers* (4)   50.0    32.0                0.610   
 4        Indiana Pacers* (5)   48.0    34.0                0.585   
 5            Miami Heat* (6)   44.0    38.0                0.537   
 6       Milwaukee Bucks* (7)   44.0    38.0                0.537   
 7    Washington Wizards* (8)   43.0    39.0                0.524   
 8        Detroit Pistons (9)   39.0    43.0                0.476   
 9     Charlotte Hornets (10)   36.0    46.0                0.439   
 10      New York Knicks (11)   29.0    53.0                0.354   
 11        Brooklyn Nets (12)   28.0    54.0                0.341   
 12        Chicago Bulls (13)   27.0    55.0                0.329   
 13        Or

> ** 9.1.extra.:** Compare your results to the pandas implementation. 

## Exercise Section 9.2: Practicing Regular Expressions.
This exercise is about developing your experience with designing your own regular expressions.

Remember you can always consult the regular expression reference page [here](https://www.regular-expressions.info/refquick.html), if you need to remember or understand a specific symbol. 

You should practice using *"define-inspect-refine-method"* described in the lectures to systematically ***explore*** and ***refine*** your expressions, and save all the patterns tried. You can download the small module that I created to handle this in the following way: 
``` python
import requests
url = 'https://raw.githubusercontent.com/snorreralund/explore_regex/master/explore_regex.py'
response = requests.get(url)
with open('explore_regex.py','w') as f:
    f.write(response.text)
import explore_regex as e_re
```

Remember to start ***broad*** to gain many examples, and iteratively narrow and refine.

We will use a sample of the trustpilot dataset that you practiced collecting yesterday.
You can load it directly into python from the following link: https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv

> **Ex. 9.2.1:** Load the data used in the exercise using the `pd.read_csv` function. (Hint: path to file can be both a url or systempath). 

>Define a variable `sample_string = '\n'.join(df.sample(2000).reviewBody)` as sample of all the reviews that you will practice on.  (Run it once in a while to get a new sample for potential differences).
Imagine we were a company wanting to find the reviews where customers are concerned with the price of a service. They decide to write a regular expression to match all reviews where a currencies and an amount is mentioned. 



In [ ]:
# [Answer to Ex. 9.2.1]

# This will be in assignment 1

> **Ex. 9.2.2:** 
> Write an expression that matches both the dollar-sign (\$) and dollar written literally, and the amount before or after a dollar-sign. Remember that the "$"-sign is a special character in regular expressions. Explore and refine using the explore_pattern function in the package I created called explore_regex. 
```python
import explore_regex as e_re
explore_regex = e_re.Explore_Regex(sample_string) # Initaizlie the Explore regex Class.
explore_regex.explore_pattern(pattern) # Use the .explore_pattern method.
```


Start with exploring the context around digits ("\d") in the data. 

In [26]:
# [Answer to exercise 9.2.2] 

# This will be in assignment 1

> **Ex.9.2.3** Use the .report() method. e_re.report(), and print the all patterns in the development process using the .pattern method - i.e. e_re.patterns 


In [ ]:
# [Answer 9.2.3]

# This will be in assignment 1

>** Ex. 9.2.4** 
Finally write a function that takes in a string and outputs if there is a match. Use the .match function to see if there is a match (hint if does not return a NoneType object - `re.match(pattern,string)!=None`).

> Define a column 'mention_currency' in the dataframe, by applying the above function to the text column of the dataframe. 
*** You should have approximately 310 reviews that matches. - but less is also alright***

> **Ex. 9.2.5** Explore the relation between reviews mentioning prices and the average rating. 

> **Ex. 9.2.extra** Define a function that outputs the amount mentioned in the review (if more than one the largest), define a new column by applying it to the data, and explore whether reviews mentioning higher prices are worse than others by plotting the amount versus the rating.

In [55]:
#[Answer to 9.2.4-5]

In [30]:
pattern = '[£$] ?[0-9]+(?:[.,][0-9]+)?|[0-9]+(?:[.,][0-9]+)? ?(?:USD|usd)|[0-9]+(?:[.,][0-9]+)? ?(?:dollars|DOLLARS)'
currency_re = re.compile(pattern)
def match_currency(string):
    return len(currency_re.findall(string))>0
df['mention_currency'] = df.reviewBody.apply(match_currency)

In [31]:
df.groupby('mention_currency').reviewRating_ratingValue.mean()

mention_currency
False    4.507275
True     2.935275
Name: reviewRating_ratingValue, dtype: float64

> **Ex. 9.2.5:** Now we write a regular expression to extract emoticons from text.
Start by locating all mouths ')' of emoticons, and develop the variations from there. Remember that paranthesis are special characters in regex, so you should use the escape character.

In [32]:
sample_string = '\n'.join(df.sample(1000).reviewBody)
emoticon_regex = e_re.ExploreRegex(sample_string)

In [40]:
pattern = '[:;][-Oo]?[()D]'
emoticon_regex.explore_pattern(pattern,context=20)

Match: :)	Context:me Aceable drivers. :)
It is simple and ha
Match: :-)	Context:ieve 5 payday loans :-)
Thank you so much f
Match: :)	Context:until we receive it :)
Bluestone Perennial
Match: :)	Context:u for your business :). 

Angry customer
Match: ;)	Context:t says in the title ;)

<3 Jani
I Book a t
Match: :)	Context:ad a wonderful time :)
Never had a problem


In [41]:
emoticon_regex.report(plot = False)

------ Pattern: [:;][-Oo]?[()D]	 Matched 6 patterns -----
------ Pattern: [:][-Oo]?[()D]	 Matched 5 patterns -----


In [42]:
emoticon_regex.pattern2n_match

{'[:;][-Oo]?[()D]': 6, '[:][-Oo]?[()D]': 5}